In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString
import pickle
import seaborn as sns
import os

import vdmlab as vdm

from load_data import get_pos, get_spikes, get_lfp
from tuning_curves_functions import get_tc_1d, get_odd_firing_idx

import info.R063d2_info as r063d2
import info.R063d3_info as r063d3
import info.R063d4_info as r063d4
import info.R063d5_info as r063d5
import info.R063d6_info as r063d6
import info.R066d1_info as r066d1
import info.R066d2_info as r066d2
import info.R066d3_info as r066d3
import info.R066d4_info as r066d4

In [ ]:
pickle_filepath = 'E:\\code\\python-vdmlab\\projects\\emily_shortcut\\cache\\pickled\\'
output_filepath = 'E:\\code\\python-vdmlab\\projects\\emily_shortcut\\plots\\sequence\\'
# pickle_filepath = 'C:\\Users\\Emily\\Code\\python-vdmlab\\projects\\emily_shortcut\\cache\\pickled\\'
# output_filepath = 'C:\\Users\\Emily\\Code\\python-vdmlab\\projects\\emily_shortcut\\plots\\sequence\\'

In [ ]:
info = r063d4

In [ ]:
print(info.session_id)
pos = get_pos(info.pos_mat, info.pxl_to_cm)
csc = get_lfp(info.good_swr[0])
spikes = get_spikes(info.spike_mat)

In [ ]:
fs = 2000
time = np.arange(0, 0.5, 1./fs)
freq = np.ones(len(time))*100
freq[int(len(time)*0.4):int(len(time)*0.6)] = 180
freq[int(len(time)*0.7):int(len(time)*0.9)] = 260
data = np.sin(2.*np.pi*freq*time)

lfp = vdm.LFP(data, time)

times, idx, butter = vdm.detect_swr_hilbert(lfp, fs=2000, thresh=(140.0, 250.0), power_thres=0.5, z_thres=0.4)

In [ ]:
plt.plot(time, data, 'g')
for start, stop in zip(idx['start'], idx['stop']):
    plt.plot(time[start:stop], data[start:stop])
plt.show

In [ ]:
import numpy as np

In [ ]:
spikes = dict()
spikes['time'] = np.array([1., 2.])
spikes['label'] = np.array(['this'])